## Q1.

Add a __setitem__ to the python linked list implementation from the lecture (this past wednesday).

In [332]:
from doctest import run_docstring_examples as dtest
import numbers
import reprlib
class LL:
    """
    >>> A = LL()  
    >>> A[0]
    Traceback (most recent call last):
        ...
    IndexError: trying to index an empty LL
    >>> A.insert_front(1)
    >>> A[0]
    1
    >>> A.insert_back(2)
    >>> A[1]
    2
    >>> A
    LL([1,...])
    >>> myll = LL.from_components([1,2])
    >>> myll[1]
    1
    >>> len(myll)
    2
    >>> myll[2]
    Traceback (most recent call last):
        ...
    IndexError: LL index out of range
    >>> myll[0:1]
    Traceback (most recent call last):
        ...
    TypeError: LL indices must be integers
    """
    @classmethod
    def from_components(cls, components):
        inst = cls(components[0])
        for c in components[1:]:
            inst.insert_front(c)
        return inst
        
    def __init__(self, head=None):
        if head is None:
            self._headNode = None
        else:
            self._headNode = [head, None]
            
    def insert_front(self, element):
        new_node = [element, None]
        new_node[1] = self._headNode
        self._headNode = new_node
        
    def insert_back(self, element):
        new_node = [element, None]
        curr_ptr = self._headNode
        while curr_ptr[1] is not None:
            curr_ptr = curr_ptr[1]
        curr_ptr[1]= new_node
        
    def __repr__(self):
        class_name = type(self).__name__
        if len(self)==0:
            components=""
        else:
            components = reprlib.repr(self[0])
        return '{}([{},...])'.format(class_name,components)

    def __len__(self):
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            return 0
        while 1:
            count = count + 1
            if curr_ptr[1] is None:
                break
            curr_ptr = curr_ptr[1]
        return count    
    
    def __getitem__(self, index):
        class_name = type(self).__name__
        if isinstance(index, numbers.Integral): 
            curr_ptr = self._headNode
            if curr_ptr==None:
                msg = 'trying to index an empty {class_name}' 
                raise IndexError(msg.format(class_name=class_name))
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if index == count:
                    return curr_ptr[0]
                if curr_ptr[1] is None:
                    msg = '{class_name} index out of range' 
                    raise IndexError(msg.format(class_name=class_name))       
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} indices must be integers' 
            raise TypeError(msg.format(class_name=class_name))
            
    def __setitem__(self, index, value):
        class_name = type(self).__name__
        if isinstance(index, numbers.Integral):            
            curr_ptr = self._headNode
            if curr_ptr==None:
                msg = 'trying to index an empty {class_name}' 
                raise IndexError(msg.format(class_name=class_name))
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if index == count:
                    curr_ptr[0] = value
                    break
                if curr_ptr[1] is None:
                    msg = '{class_name} index out of range' 
                    raise IndexError(msg.format(class_name=class_name))       
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} indices must be integers' 
            raise TypeError(msg.format(class_name=class_name))

In [334]:
my_ll = LL.from_components([1,2,3])
print(my_ll[2])
my_ll[2] = 6
my_ll[2]

1


6

## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [3]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [4]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [5]:
g = make_data(5, 10)
list(g)

[999999997.3658907,
 999999999.5357127,
 1000000000.346875,
 1000000005.136389,
 1000000004.0222701,
 1000000000.831904,
 999999999.2830019,
 1000000000.3912419,
 999999999.9604353,
 999999994.3970838,
 1000000002.3819788]

In [367]:
g = online_mean(make_data(5, 10))
list(g)

[999999999.4076003,
 1000000000.3716495,
 1000000000.4696158,
 999999999.9156064,
 1000000001.1757807,
 1000000000.8731438,
 1000000000.7923273,
 1000000000.8337649,
 1000000000.6530597,
 1000000000.5683315,
 1000000000.3384781]

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [385]:
import math
# your code here
def online_mean_dev(iterator):
    """
    Description
    ----------
    Generator to calculate running standard deviation.
    
    Parameters
    ----------
    Iterator
    
    Returns
    -------
    Yields a tuple (n, value, mean, sd)
    """
    
    # initialize values needed for calculation
    mu_old, s_old, n = 0, 0, 1
    live_mu = online_mean(iterator)
    
    # iterate over values in `iterator`
    for value in iterator:
        
        # calculate actual new values
        mu_new = next(live_mu)
        s_new = s_old + (value - mu_old)*(value - mu_new)
        
        # update 'old' values
        mu_old = mu_new
        s_old = s_new
        
        if n > 1:
            #print(s_new)
            sd = math.sqrt(s_new/(n))
            yield (n, value, mu_new, sd)
        
        # increment n
        n += 1
        

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [390]:
data_with_stats = online_mean_dev(make_data(5, 100000))

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [391]:
#your code here
def is_ok(level, t):
    """
    Description
    -----------
    Determines if the newest value of a stream
    sequence is an anomaly, in the sense of
    being `level`*SD away from the mean.
    
    Parameters
    ----------
    level: float
        width of the band of non-anomalous values.
        e.g. `level` = 1 considers anything that is
        over 1*SD from the mean an anomaly
    t: tuple
        (n, value, mu, sd)-tuple of data.
        
    Returns
    -------
    True if not anomaly, else False
    """
    n, value, mu, sd = t
    return False if abs(value - mu) >= level*sd else True

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [392]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [393]:
list(anomalies)

[]

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).